In [20]:
# First install the packages
# pip install pip install z3-solver
# pip install py_expression_eval
# pip install networkx

from py_expression_eval import Parser # Not used at the moment but might be good !
parser = Parser()
import copy
import networkx as nx
from typing import Dict, List, Set, Tuple

In [24]:
# Examples of node creation
class BranchNode():  # Just an example of a base class
    def __init__(self, id : str, condition : str):
        self.expression = condition
        self.valuesAndNext : Dict[str, str] = {} # A dictionary from expression value to next branch
        self.id = id
        
    def getVariables(self):
        return self.expression.variables()
    
    def __str__(self):
        return self.id + " " + str(self.expression)
        

In [34]:
# Functions and examples to inspect the graph at a higher level
#-------------------------------------------------
# A function to collect all variables by nodes
def getAllVariables(graph) -> set:
    setOfVariables = set()
    for node in graph.nodes:
        variablesInNode = node.getVariables()
        setOfVariables.update(variablesInNode)
    return setOfVariables
        
# A function to collect all paths in the graph...well as long as not infinite for now :) so it will work only for DAGs.
# For infinite graphs we'll consider IDA approach
def getAllPaths(graph):
    # Get all the starting nodes (in degree = 0)
    start_nodes = []
    for node in graph.nodes:
        if graph.in_degree[node] == 0:
            start_nodes.append(node)
            
    #print(start_nodes[0].name)
    
    allpaths = []
    def _findpath(graph, currNode, currPath, outAllPaths):
        currPath.append(currNode)
        if graph.out_degree[currNode] == 0: # leaf node ?
            outAllPaths.append(currPath)
        else:
            # For each successor add this node to a copy of the list and iterate on that path
            for succNode in graph.successors(currNode):            
                newCurrPath = copy.deepcopy(currPath)
                _findpath(graph, succNode, newCurrPath, outAllPaths)            
    
    # From each starting node, run a directed search path until leafs
    for snode in start_nodes:
        _findpath(graph, snode, [], allpaths)
        
    return allpaths
    
def debugPrintPaths(paths):
    for index, P in enumerate(paths):
        print("--- Path ", index, ": ")
        for node in P:
            print(node.id)
            

        
def createTestGraph_fromDict(dictSpec : Dict[str, any], graphName):
    graph = nx.DiGraph()
    graph.clear()

    
    # Step 1: Create all the node firsts and cache the inverse dictionary in the graph from node_id to node instance
    graph.graph['graphName'] = graphName
    
    nodeIdToInstance : Dict[str, BranchNode] = {}
        
    for nodeId, nodeSpec in dictSpec.items():
        assert isinstance(nodeSpec, tuple) and len(nodeSpec) > 0, f"invalid specificiation for node {nodeId}"
        nodeCond = nodeSpec[0]
        nodeInst = BranchNode(id=nodeId, condition=nodeCond)
        nodeIdToInstance[nodeId] = nodeInst
        graph.add_node(nodeId)
    
    graph.graph['nodeIdToInstance'] = nodeIdToInstance
    
    # Step 2: Create the links inside the nodes and graph
    for nodeId, nodeSpec in dictSpec.items():
        nodeSuccessorsSpec = nodeSpec[1]
        assert isinstance(nodeSuccessorsSpec, list), f"Expecting a list here for node successors desc!"
        
        
        parentNodeInst = nodeIdToInstance[nodeId]
        for nodeSucc in nodeSuccessorsSpec:
            nextNodeVal = nodeSucc[0]
            nextNodeId = nodeSucc[1]
            
            succNodeInst = nodeIdToInstance[nextNodeId]
            parentNodeInst.valuesAndNext[nextNodeVal] = nextNodeId
            
            graph.add_edge(parentNodeInst, succNodeInst)
            
    return graph

def debugInspectGraph(graph):
    print("Graph nodes: ")
    for node in graph.nodes:
        print(node)

    print("Graph edges: ")
    for edge in graph.edges:
        start = edge[0]
        end = edge[1]
        print(f"start from {start.id} end {end.id}")

    print("In Degrees: ", graph.in_degree([node for node in graph.nodes]))
    print("Out Degrees: ", graph.out_degree([node for node in graph.nodes]))
    
            
# Creating the graph now from a given dictionary 

bankLoanModel = {'node_loanTest0' : ('loan < 1000', [('True', 'term_test0'), ('False', 'node_loanTest1')]),
                 'node_loanTest1' : ('And(loan >= 1000, loan < 10000)', [('True', 'term_test0')]),
                 'term_test0' : ('term < 1000', [])
                }

graph = createTestGraph_fromDict(bankLoanModel, "Bank Loan Example")

# Let's inspect the graph...
debugInspectGraph(graph)

print("Checking all paths inside the graph !")            
paths = getAllPaths(graph)
debugPrintPaths(paths)

print("\n\nGetting all used variables inside branches ")
print(getAllVariables(graph))

Graph nodes: 
node_loanTest0
node_loanTest1
term_test0
node_loanTest0 loan < 1000
term_test0 term < 1000
node_loanTest1 And(loan >= 1000, loan < 10000)
Graph edges: 
start from node_loanTest0 end term_test0
start from node_loanTest0 end node_loanTest1
start from node_loanTest1 end term_test0
In Degrees:  [('node_loanTest0', 0), ('node_loanTest1', 0), ('term_test0', 0), (<__main__.BranchNode object at 0x7ffa286100b8>, 0), (<__main__.BranchNode object at 0x7ffa286100f0>, 2), (<__main__.BranchNode object at 0x7ffa28610128>, 1)]
Out Degrees:  [('node_loanTest0', 0), ('node_loanTest1', 0), ('term_test0', 0), (<__main__.BranchNode object at 0x7ffa286100b8>, 2), (<__main__.BranchNode object at 0x7ffa286100f0>, 0), (<__main__.BranchNode object at 0x7ffa28610128>, 1)]
Checking all paths inside the graph !
--- Path  0 : 


AttributeError: 'str' object has no attribute 'id'

In [ ]:
# Z3 now 

    

In [ ]:
# TODO:
# Drawing graphs
# https://networkx.org/documentation/stable/tutorial.html#drawing-graphs
# TODO: YAML integration !

print(A)